# Zaawansowane sieci neuronowe w detekcji sentymentu

Na dzisiejszych laboratoriach skupimy się na wykorzystaniu zaawansowanych architektur sieci neuronowych do problemu wykrywania sentymentu (emocji: pozytywnych i negatywnych), które zawarte są w tekstach.

Ponieważ implementacja sieci LSTM i GRU jest dość trudna/czasochłonna - wykorzystamy gotowy framework, który pozwoli nam na zdefiniowanie i wyuczenie sieci neuronowej na wysokim poziomie - **Keras**.

Ocenę sentymentu przeprowadzimy na gotowym zbiorze recenzji z portalu IMDB, który jest już odpowiednio przeprocesowany i posiada zdefiniowany oczekiwany sentyment dla każdego tekstu (a więc dla którego bez wysiłku możemy uruchomić algorytmy klasyfikacji i je ocenić). Zaczynajmy!

## Dane do uczenia

Poniższy fragment kodu pobiera dane do uczenia. Funkcja imdb.load_data() ładuje zarówno zbiór uczący (wektory cech, oraz etykiety), jak i analogiczny zbiór testowy. 

Poniżej wyświetlony na ekranie jest jeden z przykładów uczących oraz przypisana do niego etykieta.

Widzimy, że tekst reprezentowany jest sekwencją liczb. Co one oznaczają?
Każda liczba reprezentuje słowo (jest identyfikatorem słowa), identyfikatory posortowane są względem częstości występowania słów, zatem słowo o identyfikatorze 10 występuje w korpusie częśćiej niż słowo o identyfikatorze 11.
Dodatkowo wprowadzone są specjalne znaczniki BOS - początek zdania i EOS - koniec zdania. Oba równiez reprezentowane są w formie liczbowej.

Pamiętamy z jednych z pierwszych laboratoriów, że duża wielkość słownika jest problematyczna. Dobrym pomysłem jest często odrzucenie najrzadziej wystepujących słów, ponieważ one nie mają wielkiego znaczenia (Kiedy uczymy się nowego języka - często nie rozumiemy pojedynczych słów, ale znajomość pozostałych sprawia, że jesteśmy w stanie zrozumieć sens tekstu). Aby ograniczyć rozmiar słownika, w funkcji load_data() możemy zadać parametr num_words o określonej wartości. Wartość ta, mówi nam ile najczęściej występujących słów bierzemy pod uwagę. Wszystkie rzadsze słowa - reprezentowane są zbiorczo taką samą wartością liczbową oznaczającą nieznany token (Unknown token).

Inną ważną kwestią jest długość recenzji - każda z nich może składać się z innej liczby słów. O ile sieci rekurencyjne są teoretycznie w stanie poradzić sobie z sekwencjami o różnej długości, to w praktyce optymalizacje wymagają, aby sekwencje były reprezentowane poprzez taką samą długość wektora cech. Aby wyrównać liczbę cech na wejściu stosuje się tzw. padding do określonej długości. Jeśli wektor cech recenzji jest dłuższy niż zadany padding - zostaje on ucięty, jeśli zaś jest krótszy - dodawane są cechy o wartości 0, aby dopełnić długości.

**Zadanie 1 (1.25 punktu)**: Poniższy kod pobiera dane z IMDB ograniczając liczbę słów w słowniku do 10000. 
Chcielibyśmy przyjrzeć się danym oraz zastosować na nich padding. Aby to zrobić - wykonajmy następujące kroki:
<ol>
    <li>Sprawdźmy i wyświetlmy średnią długość wektora w x_train - pozwoli nam to sprawdzić ile średnio słów jest w recenzji</li>
    <li>Sprawdźmy i wyświetlmy odchylenie standardowe wektora x_train - pozwoli nam to określić jak wygląda rozrzut wartości od średniej</li>
    <li>Stosując funkcję pad_sequences z kerasa (zaimportowana w pierwszej linijce) - nadpiszmy zbiory x_train i x_test tak, aby każdy wektor miał długość 500 (https://keras.io/preprocessing/sequence/). Wybrana długość wynika z analizy z poprzednich podpunktów (średnia i odch. std.). Jak teraz wygląda średnia długość i odchylenie std?</li>
    <li>Nasz model będziemy weryfikować na zbiorze testowym z użyciem miary accuracy (jaki % podjętych przez klasyfikator decyzji jest poprawnych). Warto sprawdzić jak wygląda rozkład etykiet w zbiorze testowym. Sprawdź: jaki procent zbioru testowego stanowią etykiety o wartości 1? jaki procent zbioru testowego stanowią etykiety o wartości 0?
</ol>

In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.datasets import imdb
import numpy as np

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

print(x_train[0]) # pokaż wektor cech dla pierwszej recenzji
print(y_train[0]) # pokaż etykietę (1 = sentyment pozytywny; 0 = sentyment negatywny)

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [32]:
average_x_len = np.mean([len(x) for x in x_train]) # TODO: oblicz średnią liczbę cech w wektorach x_train (możesz wykorzystać numpy)
stddev_x_len = np.std([len(x) for x in x_train]) # TODO: oblicz odchylenie standardowe po x_train (możesz wykorzystać numpy)

x_train = pad_sequences(x_train, 500) # TODO: zastosuj padding do 500 tokenów (wskazówka: zobacz na listę importowanych funkcji)
x_test = pad_sequences(x_test, 500)   # TODO: zastosuj padding do 500 tokenów

padded_average_x_len = np.mean([len(x) for x in x_train]) # TODO: oblicz średnią liczbę cech w wektorach x_train po paddingu
padded_stddev_x_len = np.std([len(x) for x in x_train]) # TODO: oblicz odchylenie standardowe po x_train po paddingu

counts = dict(zip(*np.unique(y_test, return_counts=True)))
count_positive = counts[1] # TODO: ile elementów testowych ma przypisany sentyment pozytywny
count_negative = counts[0] # TODO: ile elementów testowych ma przypisany sentyment negatywny

print("Przed paddingiem. Średnia długość wektora: {ave_len}; odchylenie std: {std_dev}".format(
    ave_len=average_x_len, std_dev=stddev_x_len))

print("Po paddingu. Średnia długość wektora: {ave_len}; odchylenie std: {std_dev}".format(
    ave_len=padded_average_x_len, std_dev=padded_stddev_x_len))

print("W zbiorze testowym jest {pos} elementów o pozytywnym sentymencie i {neg} elementów o negatywnym. Sentyment pozytywny stanowi {percentage}% zbioru.".format(
pos=count_positive, neg=count_negative, percentage = 100.0*(count_positive)/(count_positive + count_negative)))

Przed paddingiem. Średnia długość wektora: 500.0; odchylenie std: 0.0
Po paddingu. Średnia długość wektora: 500.0; odchylenie std: 0.0
W zbiorze testowym jest 12500 elementów o pozytywnym sentymencie i 12500 elementów o negatywnym. Sentyment pozytywny stanowi 50.0% zbioru.


## Przykładowa prosta sieć w Keras.
Poniżej znajdziecie przykład kodu, którzy tworzy sieć dwuwarstwową o:
<ol>
<li>100 wejściach</li>
<li>warstwie ukrytej z 64 neuronami o aktywacji ReLU</li>
<li>warstwie wyjściowej z 1 neuronem o aktywacji sigmoidalnej</li>
</ol>
Ten kod będzie szablonem dla kolejnych zdań. Uruchom go i sprawdź jak prosta sieć działa 

$ReLU(x) = max(0, x)$, 

In [33]:
import numpy as np
np.random.seed(1337) # for reproducibility

from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, LSTM, GRU, Conv1D, MaxPooling1D


model = Sequential() # sequential = sieć jako lista warstw, dodajemy warstwy metodą .add() (jak w standardowej liście)
model.add(Dense(units=64, input_dim=500, activation='relu')) # dodajemy warstwę Dense (gęstą). Dense oznacza, że wszystkie wejścia (w tym przypadku 100) połączone są z neuronami warstwy w sposób każdy z każdym (każdy neuron z poprzedniej warstwy połączony z każdym neuronem warstwy następnej, tak jak to robiliśmy na poprzednich laboratoriach)
model.add(Dense(units=1, activation='sigmoid')) # rozmiar wejścia zdefiniować musimy tylko w pierwszej warstwie (definiujemy ile jest cech na wejściu). Ponieważ model wie jakie są rozmiary poprzednich warstw - może w sposób automatyczny odkryć, że opprzednia warstwa generuje 64 wyjścia

model.compile(loss='binary_crossentropy', # budujemy model! ustawiamy funkcję kosztu - mamy klasyfikację z dwiema etykietami, więc stosujemy 'binary_crossentropy'
              optimizer='adam',  # wybieramy w jaki sposób sieć ma się uczyć
              metrics=['accuracy']) # i wybieramy jaka miara oceny nas interesuje


model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test)) # uczymy model na zbiorze treningowym, weryfikujemy na testowym, epochs - oznacza ile przejść po wszystkich przykładachw zbiorze uczącym powinno się wykonać.

loss, accuracy = model.evaluate(x_test, y_test, batch_size=128) # ostateczna ewaluacja wyuczonego modelu
print("Trafność klasyfikacji to: {acc}%".format(acc=accuracy*100)) 

Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 5s 190us/step - loss: 100.8503 - accuracy: 0.5010 - val_loss: 6.2868 - val_accuracy: 0.5005
Epoch 2/5
25000/25000 [==============================] - 3s 117us/step - loss: 2.7351 - accuracy: 0.5107 - val_loss: 1.7895 - val_accuracy: 0.4983
Epoch 3/5
25000/25000 [==============================] - 3s 118us/step - loss: 1.1170 - accuracy: 0.5071 - val_loss: 1.2156 - val_accuracy: 0.5014
Epoch 4/5
25000/25000 [==============================] - 3s 119us/step - loss: 0.8237 - accuracy: 0.5120 - val_loss: 1.0892 - val_accuracy: 0.4982
Epoch 5/5
25000/25000 [==============================] - 0s 11us/step
Trafność klasyfikacji to: 49.696001410484314%


Jak widzimy, sieć generuje trafność na poziomie 50%. Ponieważ zarówno etykieta "1", jak i "0" w zbiorze testowym stanowią połowę - wiemy, że ten klasyfikator nie jest najlepszy (Taką samą trafność będą miały klasyfikatory: zwracające zawsze etykietę 0, zwracajace zawsze etykietę 1 oraz zwracające decyzje losowe).

Czy jesteśmy w stanie coś z tym zrobić? 
Tak. Nasza poprzednia sieć próbowała uczyć się z listy identyfikatorów słów, to stosunkowo kiepska reprezentacja, ale pamiętamy, że całkiem nieźle sprawowały się tzw. Embeddingi. Szczęśliwie - keras udostępnia warstwy uczące się embeddingów z reprezentacji takiej, którą dotychczas podawaliśmy na wejściu.



**Zadanie 2 (1.25 punktu) - Wykorzystanie embeddingów w sieci feed forward**
Widząc w jaki sposób dodawane są kolejne warstwy w Kerasie (model.add(...)), przerób architekturę istniejącej sieci w następujący sposób:

<ol>
    <li>Pierwsza warstwa: Warstwa Embedding (https://keras.io/layers/embeddings/), ustaw długość generowanego wektora na 32, długość wejścia - taka jak wynika to z paddingu - 500, a także rozmiar słownika zgodny z tym co wybraliśmy przy pobieraniu danych (10000)</li>
    <li>Druga warstwa: Flatten (https://keras.io/layers/core/); Zauważmy, że warstwa ucząca embeddingi - Embedding - zamienia nam każdy indentyfikator z wektora wejściowego na wektor o zadanej liczbie wymiarów. Każde słowo reprezentowane jest teraz nie pojedynczą liczbą a pojedynczym wektorem. Kiedy złożymy embeddingi wszystkich słów otrzymamy macierz wielkości: liczba słów x rozmiar embeddingu. Warstwa Flatten nie robi nic poza tym, że bierze taką macierz i zamienia znów na wektor poprzez połączenie ze sobą wszystkich wektorów embeddingów w jeden wielki wektor (ustawiając je w jednym wymiarze jeden za drugim) </li>
    <li>Trzecia warstwa: klasyczna warstwa Dense (https://keras.io/layers/core/) np. z 64 neuronami i aktywacją relu
    <li>Czwarta warstwa (wyjściowa): klasyczna warstwa Dense z 1 neuronem (generującym prawdopodobieństwo pozytywnego sentymentu) i aktywacją sigmoidalną (sigmoid)
</ol>
Parametry kompilacji, sposób uczenia i ewaluacji możesz pozostawić bez zmian. Czy trafność klasyfikacji wzrosła?


In [34]:
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32, input_length=500))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)
loss, accuracy = model.evaluate(x_test, y_test)
print("Trafność klasyfikacji to: {acc}%".format(acc=accuracy*100)) 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 1s 52us/step - loss: 3.7444 - accuracy: 0.3882 - val_loss: 0.4727 - val_accuracy: 0.7884
Epoch 2/2
25000/25000 [==============================] - 1s 43us/step
Trafność klasyfikacji to: 64.47200179100037%


## Architektury rekurencyjne

Aby zamodelować sieć rekurencyjną LSTM bądź GRU - możemy użyć dedykowanych warstw przygotowanych przez autorów Kerasa.


**Zadanie 3 (1.25 punktu): Sieć rekurencyjna GRU i LSTM**
Aby stworzyć taką sieć utwórz model z następującymi warstwami:

<ol>
    <li>Warstwa Embedding, analogicznie do poprzednich zadań. Rozmiar wektora embeddingów ustawmy na 32</li>
    <li>Warstwa LSTM (https://keras.io/layers/recurrent/) - Warstwa sieci rekurencyjnej - nie potrzebuje wcześniejszego spłaszczenia warstwą Flatten. Ustawmy rozmiar tej warstwy na 32. Ponadto ustawmy parametry dropout i recurrent_dropout na 0.2 (parametr regularyzacyjny zabezpieczający przet przeuczeniem)</li>
    <li>Warstwa Dense (wyjściowa) - Warstwa o aktywacji sigmoidalnej z 1 neuronem</li>
</ol>

Po uruchomieniu sieci wykorzystującej LSTM - zamień warstwę LSTM na GRU (https://keras.io/layers/recurrent/) z takimi samymi parametrami - czy sieć uczy się lepiej? Co z czasem uczenia?

In [41]:
import time

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32, input_length=500))
model.add(GRU(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
start_time = time.time()
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)
end_time = time.time()
loss, accuracy = model.evaluate(x_test, y_test)
print("Trafność klasyfikacji to: {acc}%".format(acc=accuracy*100)) 
print("Czas treningu: {t}".format(t=end_time - start_time))

# LSTM czas: 385s, accuracy: 82.46%
# GRU  czas: 469s, accuracy: 68.72%
# dziwne, GRU jest prostsze, wiec powinno sie szybciej uczyc...
# no i ten eksplodujacy loss...

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 234s 9ms/step - loss: 1136248180687071758835209207808.0000 - accuracy: 0.6733 - val_loss: 0.6289 - val_accuracy: 0.6834
Epoch 2/2
 1408/25000 [>.............................] - ETA: 3:27 - loss: 56922502.3258 - accuracy: 0.7663

KeyboardInterrupt: ignored

Jak widzimy siec rekurencyjna daje niższe rezultaty niż sieć feedforward. Dlaczego? 
Sieci rekurencyjne (LSTM i GRU) dają w wielu zadaniach wyniki, które są najlepsze możliwe. Jednakże trening takiej sieci trwa bardzo długo. Gdybyśmy odpowiednio dobrali liczbę warstw i parametry sieci - prawdopodobnie otrzymalibyśmy najlepsze rezultaty ze wsystkich porównywanych architektur - niestety za cenę czasu, który na laboratoriach jest ograniczony.

Ponieważ obliczenia w niektórych architekturach są bardzo intensywne, bardzo popularnym jest wykonywanie tych obliczeń nie na procesorze, a na karcie graficznej. W przypadku posiadania dobrej karty graficznej, szybkość przetwarzania będzie dużo większa.

## Architektury konwolucyjne

Sieci konwolucyjne w Kerasie zostały już wykorzystane na laboratoriach ze sztucznej inteligencji. Wtedy - używaliśmy ich do detekcji czy obrazek przedstawiony na wejściu sieci reprezentował kota czy psa.

Okazuje się, że problemach klasyfikacji tekstu sieci konwolucyjne (CNN - convolutional neural network) radzą sobie również bardzo dobrze (dają niezłe rezultaty, a czas ich uczenia jest zazwyczaj dużo niższy niż sieci rekurencyjnych)!

Sprawdźmy jakie rezultaty otrzymamy zaprzęgając sieć konwolucyjną do naszego problemu:

**Zadanie 4 (1.25 punktu)**: Przygotuj sieć konwolucyjną wg. następującego schematu:
<ol>
    <li>Warstwa pierwsza - Warstwa Embedding, analogiczna do poprzednich zadań </li>
    <li>Warstwa druga - konwolucja jednowymiarowa. Użyj warstwy Conv1D (https://keras.io/layers/convolutional/) używając 32 filtrów, rozmiaru tzw. kernela = 3, padding ustawmy na 'same', a jako funkcję aktywacji 'relu' </li>
    <li>Warstwa trzecia - MaxPooling1D (https://keras.io/layers/convolutional/), ustawmy rozmiar pool_size na 2 </li>
    <li>Warstwa czwarta - Flatten - Znów - zamieniamy macierz będącą efektem operacji konwolucji na wektor </li>
    <li>Warstwa piąta - Dense, 250 neuronów z aktywacją relu</li>
    <li>Warstwa szósta (wyjściowa) - Dense, 1 neuron wyjściowy z aktywacją sigmoidalną (sigmoid)</li>
</ol>


In [46]:
model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=32, input_length=500))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=2, batch_size=128)
loss, accuracy = model.evaluate(x_test, y_test)
print("Trafność klasyfikacji to: {acc}%".format(acc=accuracy*100)) 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 2s 69us/step - loss: 0.4982 - accuracy: 0.7255 - val_loss: 0.3157 - val_accuracy: 0.8640
Epoch 2/2
25000/25000 [==============================] - 1s 49us/step
Trafność klasyfikacji to: 87.30800151824951%
